In [1]:
import minsearch
import json
import os
from openai import OpenAI

In [15]:
os.environ["OPENAI_API_KEY"] = "sk-"

In [16]:
api_key = os.environ.get('OPENAI_API_KEY')

In [17]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [18]:
documents =[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [19]:
client = OpenAI(api_key=api_key)

In [20]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [21]:
index.fit(documents)

In [22]:
def search(query):
    boost = {'question': 3.0, 'section':0.5}

    results = index.search(query=query, filter_dict = {'course': 'data-engineering-zoomcamp'},
             boost_dict = boost, 
             num_results = 5
            
            )
    return results
    

In [23]:
def build_prompt(query, search_results):
    prompt_template="""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt
    

In [24]:
def llm(prompt):
    response = client.chat.completions.create(model="gpt-4o", 
                                          messages = [{"role":"user", "content":prompt}])
    
    return response.choices[0].message.content

In [25]:
query = 'how do i run kafka'
def rag(query):
    search_results =search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    print (answer)

In [26]:
rag(query)

To run Kafka, based on the context provided for Java Kafka, you need to execute the following command in the terminal from your project directory:

```sh
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

Replace `<jar_name>` with the actual name of your JAR file.

If you are working with a Python Kafka producer and facing issues with the Kafka module, you should set up a virtual environment and install the required packages:

1. Create and activate your virtual environment:

```sh
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
```

For Windows, the command to activate the environment is slightly different:

```sh
env\Scripts\activate
```

3. Once your virtual environment is set up and activated, you can run your Python Kafka producer. If you need to deactivate the virtual environment, use:

```sh
deactivate
```

Make sure all your Docker images, if used, are up and running before any attempts to run the K

In [32]:
rag("what is the ideal timeline to finish this course")

The ideal timeline to finish this course has not been explicitly provided in the FAQ database. However, it is mentioned that participants are expected to spend about 5 - 15 hours per week on the course. Using this information, you can estimate your own timeline based on the total duration of the course and your weekly availability. 

For instance, if the course is designed to be completed in 10 weeks and you allocate around 10 hours per week, you would likely align with the course schedule. Adjust accordingly if you plan to spend more or fewer hours per week.


In [28]:
print(answer)

To run Kafka components like producer, consumer, or KStreams in the terminal, you can follow these steps based on the language you are using.

### Java Kafka:
In the project directory, run:
```
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

### Python Kafka:
1. **Create a virtual environment** (only once):
    ```
    python -m venv env
    source env/bin/activate
    pip install -r ../requirements.txt
    ```
   For subsequent activations (every time you need the virtual env):
    ```
    source env/bin/activate
    ```
   To deactivate:
    ```
    deactivate
    ```
   Note: On Windows, the activation command is slightly different:
    ```
    env\Scripts\activate
    ```
2. **Ensure Docker images are up and running** before running the Python files.

By adhering to these steps, you should be able to successfully run your Kafka components.


In [2]:
from elasticsearch import Elasticsearch

In [3]:
es_client = Elasticsearch('http://localhost:9200')

In [4]:
es_client

<Elasticsearch(['http://localhost:9200'])>

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [27]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:21<00:00, 43.10it/s]


In [29]:
query

'how do i run kafka'

In [37]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index = index_name, body= search_query)
    result_docs  = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [38]:
elastic_search(query)

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repositor

In [39]:
def elastic_rag(query):
    search_results =elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    print (answer)

In [40]:
elastic_rag(query)

To run Kafka, follow these instructions depending on whether you are working with Java or Python:

**Java Kafka:**
1. Navigate to your project directory.
2. Use the following command to run the necessary Java file:
   ```bash
   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
   ```

**Python Kafka:**
1. Create a virtual environment if you haven't already:
   ```bash
   python -m venv env
   source env/bin/activate  # For MacOS/Linux
   # For Windows:
   # env\Scripts\activate
   ```
2. Install the required packages from `requirements.txt`:
   ```bash
   pip install -r requirements.txt
   ```
3. Make sure that Docker images are up and running if your setup requires them.

By following these steps, you should be able to get Kafka up and running for both Java and Python environments as required by the course material.
